In [38]:
import sys
sys.path.append("../..")  # Adjust the path to import canopus

from qiskit import QuantumCircuit, qasm2
from qiskit.circuit.library import QFT
from qiskit.transpiler import PassManager
import uuid
import os

import canopus


def canopus_pass(qc, topology, isa='cx'):
    if topology == "chain":
        coupling_map = canopus.utils.gene_chain_coupling_map(qc.num_qubits)
    elif topology == "hhex":
        coupling_map = canopus.utils.gene_hhex_coupling_map(qc.num_qubits)
    elif topology == "square":
        coupling_map = canopus.utils.gene_square_coupling_map(qc.num_qubits)
    else:
        raise ValueError(f"Unsupported topology: {topology}")
    
    backend = canopus.CanopusBackend(coupling_map, isa)
    qc_mapped = PassManager(canopus.CanopusMapping(backend)).run(qc)
    return canopus.rebase_to_canonical(qc_mapped)



def toqm_pass(qc, topology):
    TOQM_FLAGS = '-defaults -latency Latency_1_2_6 -expander GreedyTopK 10 -queue TrimSlowNodes 2000 1000 -nodeMod GreedyMapper -retain 1'

    input_fname = f"{uuid.uuid4().hex}.qasm"
    qasm2.dump(qc, input_fname)

    if topology == "chain":
        coupling_file = '../../configs/chain.txt'
        coupling_map = canopus.utils.gene_chain_coupling_map(qc.num_qubits)
    elif topology == "hhex":
        coupling_file = '../../configs/hhex.txt'
        coupling_map = canopus.utils.gene_hhex_coupling_map(qc.num_qubits)
    elif topology == "square":
        coupling_file = '../../configs/square.txt'
        coupling_map = canopus.utils.gene_square_coupling_map(qc.num_qubits)
    else:
        raise ValueError(f"Unsupported topology: {topology}")
    
    # create a tmp file
    output_fname = f"{uuid.uuid4().hex}.qasm"

    coupling_fname = f'./tmp_{uuid.uuid4().hex}.txt'
    graph = coupling_map.graph.to_undirected()
    with open(coupling_fname, 'w') as f:
        f.write('{} {}\n'.format(graph.num_nodes(), graph.num_edges()))
        for src, dst in graph.edge_list():
            f.write('{} {}\n'.format(src, dst))

    os.system(f'../mapper {input_fname} {coupling_file} {TOQM_FLAGS} > {output_fname}')
    os.remove(coupling_fname)  # Clean up temporary coupling file

    # Replace 'swp ' with 'swap ' in the output file
    with open(output_fname, 'r') as f:
        content = f.read()
    content = content.replace('swp ', 'swap ')
    with open(output_fname, 'w') as f:
        f.write(content)

    qc_toqm = QuantumCircuit.from_qasm_str(content)

    # remove the temporary files
    os.remove(input_fname)
    os.remove(output_fname)
    return canopus.rebase_to_canonical(qc_toqm)


qft6 = QFT(6, do_swaps=False).decompose()
qft6_cx = canopus.logical_optimize(qft6)
qft6_can = canopus.rebase_to_tk2(qft6)

qft16 = QFT(10, do_swaps=False).decompose()
qft16_cx = canopus.logical_optimize(qft16)
qft16_can = canopus.rebase_to_tk2(qft16)


/Users/anan/git-projects/quantum/canopus/experiments/analysis/../../canopus/utils.py:213: UserWarning: !!!!!! Unsupported pytket circuit type: {OpType.TK1, OpType.CX} for native conversion
  warnings.warn(


In [39]:
qc_toqm = toqm_pass(qft16_cx, 'chain')
# qc_canopus = canopus_pass(qft16_can, 'chain')

//Note: initial mapping (logical qubit at each location): 8, 9, 7, 6, 5, 4, 3, 2, 1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 
//Note: initial mapping (location of each logical qubit): 9, 8, 7, 6, 5, 4, 3, 2, 0, 1, 


In [40]:
canopus.utils.print_circ_info(qc_toqm)
canopus.utils.print_circ_info(qc_canopus)

+------------+-----------+--------------+-------+----------+
| num_qubits | num_gates | num_2q_gates | depth | depth_2q |
+------------+-----------+--------------+-------+----------+
|     27     |    141    |      46      |   37  |    18    |
+------------+-----------+--------------+-------+----------+
+------------+-----------+--------------+-------+----------+
| num_qubits | num_gates | num_2q_gates | depth | depth_2q |
+------------+-----------+--------------+-------+----------+
|     16     |    428    |     164      |   94  |    52    |
+------------+-----------+--------------+-------+----------+


In [34]:
qc_toqm.draw(fold=10000)

global phase: 1.9607
             ┌────────────┐       ┌────────────────┐┌─────────────────────────┐                                              ┌────────────────┐           ┌───────────┐           ┌─────────────────┐    ┌─────────────┐    ┌───────────────────┐      ┌────────────┐        ┌────────────────┐                                                                                   ┌─────────────────┐          ┌─────────────┐      ┌───────────────────┐     ┌────────────┐       ┌────────────────┐         ┌────────────────┐                                                                                                                                                                           ┌─────────────────────┐ ┌──────────────────┐    ┌────────────────┐    ┌─────────────────────────────┐                                                                                                                                                                                                                                                                                                                                   ┌────────────────────┐ ┌───────────────────────────┐                                                ┌────────────────────┐ ┌───────────────────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              ┌───────────────────┐      ┌────────────┐    ┌─────────────────────┐┌─────────────────────┐ ┌───────────────────┐     ┌─────────────────┐                                                                                                                                                                ┌────────────────────┐┌──────────────────────────┐                                                                                                                                                       ┌────────────────┐         ┌────────────┐       ┌───────────────────┐┌───────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       ┌───────────────────┐     ┌────────────┐     ┌──────────────────┐  ┌───────────────────────┐                                         ┌─────────────────┐ ┌───────────────────┐┌───────────────────┐  ┌────────────┐    ┌──────────────────┐  ┌───────────────────────┐                                                                                       ┌─────────────────┐   ┌────────────────────┐┌────────────────┐┌─────────────────────────┐                       ┌───────────────────┐      ┌──────────┐               